# Homework 7 (30 marks)
Create a copy of the notebook to start answering the questions. Name your notebook in the format HW7_ZHANG_DAIXI.ipynb to facilitate the grading process.

Answer all the questions, test your code to ensure there are no errors and the results are as expected. Once you have answered all the questions, save the final copy, then go to File-> click on Download.ipynb. Once the local copy has been downloaded, submit your file on Blackboard under the corresponding assignment section. Also provide us a link to your notebook during submission.

NOTE: Please give the TAs the permission to access your notebooks through the links you have provided during submission.

The due date of this homework is 04/23/2021 (Friday).

Please ensure you follow all the steps mentioned in the homework.

You can submit your solutions any number of times until the deadline.

The datasets used in this homework can be found in the google drive link below -

https://drive.google.com/drive/folders/1NxCh4X7u7wVo5aHojxjLNs9wC7B7zJhb?usp=sharing

Follow the necessary steps to import data to test your code. You can use any method to read your data in the notebook. We will not be grading the methods you use. We will only grade the code from the point where you read the dataset into a pandas dataframe - (pd.read_csv('file_name'))

Import all the libraries you require in the cell below.


In [176]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
# Enter your code here
# Data Preparetion and Model Assessment
from sklearn.metrics import classification_report, recall_score, precision_score, accuracy_score
from sklearn.model_selection import train_test_split

# Models
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

## Implement the Ensemble methods learnt in class and compare their accuarcies.

The dataset you are going to be using for homework is the **Wisconsin Breast Cancer dataset (cancer.csv)**

The dataset contains a total number of 10 features labeled in either benign or malignant classes. The features have 699 instances out of which 16 feature values are missing. The dataset only contains numeric values.

Attribute Information:

1. Sample code number: id number
2. Clump Thickness: 1 - 10
3. Uniformity of Cell Size: 1 - 10
4. Uniformity of Cell Shape: 1 - 10
5. Marginal Adhesion: 1 - 10
6. Single Epithelial Cell Size: 1 - 10
7. Bare Nuclei: 1 - 10
8. Bland Chromatin: 1 - 10
9. Normal Nucleoli: 1 - 10
10. Mitoses: 1 - 10
11. Class: (2 for benign, 4 for malignant) (**target variable**)

For more information: https://archive.ics.uci.edu/ml/datasets/breast+cancer+wisconsin+(original)

### 1. Read the dataset into variable called '**data**' (1 mark)

In [177]:
pd.set_option('display.max_columns', 100)
# Enter your code here
data = pd.read_csv("/content/sample_data/cancer.csv")
data.head()

,Sample Code Number,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bland Chromatin,Bare Nuclei,Normal Nucleoli,Mitosis,Class
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2


### **Preprocessing**: Data needs to be preprocessed before implementing ensemble methods. It is done for you here. 
### Run the below code first and then answer the questions from 2 - 7.

#### Deleting unnecessary columns: The column "Sample code number" is just an indicator and it's of no use in the modeling. So, let's drop it:


In [178]:
data.drop(['Sample Code Number'],axis = 1, inplace = True)


#### Handling missing values : 
As mentioned earlier, the dataset contains missing values. The column named "Bland Chromatin" contains them. The missing values are represneted as "?". 

Replace those "?"s with 0's and impute them with Mean Imputation

In [179]:
data['Bland Chromatin']

0       1
1      10
2       2
3       4
4       1
       ..
694     2
695     1
696     3
697     4
698     5
Name: Bland Chromatin, Length: 699, dtype: object

In [180]:
data.replace('?',0, inplace=True)

In [181]:
# Convert the DataFrame object into NumPy array otherwise you will not be able to impute
values = data.values
# Now impute it
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputedData = imputer.fit_transform(values)

#### Normalizing the data:
Ranges of the features of the dataset are not the same. This may cause a problem. A small change in a feature might not affect the other. To address this problem, normalize the ranges of the features to a uniform range, in this case, 0 - 1.

In [182]:
scaler = MinMaxScaler(feature_range=(0, 1))
normalizedData = scaler.fit_transform(imputedData)
cols = ['Clump Thickness', 'Uniformity of Cell Size', 'Uniformity of Cell Shape', 'Marginal Adhesion', 'Single Epithelial Cell Size', 'Bland Chromatin', 'Bare Nuclei', 'Normal Nucleoli', 'Mitosis','Class']
normalizedData = pd.DataFrame(normalizedData, columns=cols)
print(normalizedData.head())

   Clump Thickness  Uniformity of Cell Size  Uniformity of Cell Shape  \
0         0.444444                 0.000000                  0.000000   
1         0.444444                 0.333333                  0.333333   
2         0.222222                 0.000000                  0.000000   
3         0.555556                 0.777778                  0.777778   
4         0.333333                 0.000000                  0.000000   

   Marginal Adhesion  Single Epithelial Cell Size  Bland Chromatin  \
0           0.000000                     0.111111              0.1   
1           0.444444                     0.666667              1.0   
2           0.000000                     0.111111              0.2   
3           0.000000                     0.222222              0.4   
4           0.222222                     0.111111              0.1   

   Bare Nuclei  Normal Nucleoli  Mitosis  Class  
0     0.222222         0.000000      0.0    0.0  
1     0.222222         0.111111      0.0

### Data preprocessing is done and now you will answer the below questions using the **normalizedData**: 

### 2. Split the data into test and training data with test size - 30%. Compute the baseline classification accuracy for X_train. (3 marks)

In [183]:
normalizedData.head()

,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bland Chromatin,Bare Nuclei,Normal Nucleoli,Mitosis,Class
0,0.444444,0.000000,0.000000,0.000000,0.111111,0.1,0.222222,0.000000,0.0,0.0
1,0.444444,0.333333,0.333333,0.444444,0.666667,1.0,0.222222,0.111111,0.0,0.0
2,0.222222,0.000000,0.000000,0.000000,0.111111,0.2,0.222222,0.000000,0.0,0.0
3,0.555556,0.777778,0.777778,0.000000,0.222222,0.4,0.222222,0.666667,0.0,0.0
4,0.333333,0.000000,0.000000,0.222222,0.111111,0.1,0.222222,0.000000,0.0,0.0


In [184]:
# Enter your code here
X = normalizedData.iloc[:,:9].values
y = normalizedData["Class"].values
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3, stratify=y)


In [185]:
dummy_model = DummyClassifier(strategy='most_frequent')
dummy_model.fit(X_train,y_train)
baseline_pred = dummy_model.predict(X_test)
baseline_accuracy = accuracy_score(y_test,baseline_pred)
print("Baseline Accuracy is:", baseline_accuracy)

Baseline Accuracy is: 0.6571428571428571


### 3.  Bagging : Build a generic Bagging ensemble and print the accuracy (4 marks)
---


Hyperparameters:

Base estimator = DecisionTreeClassifier

n_estimators = 10

random_state = 42

---


In [186]:
# Generic Bagging model
# Enter your code here
decision_tree_model = DecisionTreeClassifier()
bagging_model = BaggingClassifier(base_estimator= decision_tree_model, n_estimators=10, random_state=42)
bagging_model.fit(X_train,y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                        class_weight=None,
                                                        criterion='gini',
                                                        max_depth=None,
                                                        max_features=None,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=1,
                                                        min_samples_split=2,
                                                        min_weight_fraction_leaf=0.0,
                                                        presort='deprecated',
                                                        random_state=None,


In [187]:
Bagging_pred = bagging_model.predict(X_test)
Bagging_Model_accuracy = accuracy_score(y_test,Bagging_pred)
print("Bagging Model Accuracy is:", Bagging_Model_accuracy)
#print(classification_report(y_test,Bagging_pred))

Bagging Model Accuracy is: 0.9476190476190476


### 4. RandomForest : (7 marks)
#### a) Build a Random Forest model and print the accuracy (4 marks)
---

Constructor arguments: 


n_estimators = 100, max_features = 7 and random_state = 42 


---




In [188]:
# Random Forest model
# Enter your code here
rf_model = RandomForestClassifier(n_estimators = 100, max_features = 7 , random_state = 42) 
rf_model.fit(X_train,y_train)
rf_pred = rf_model.predict(X_test)
rf_Model_accuracy = accuracy_score(y_test,rf_pred)
print("Random Forest Accuracy is:", rf_Model_accuracy )
#print(classification_report(y_test,rf_pred))

Random Forest Accuracy is: 0.9523809523809523


####  b) Calculate the top 3 important features for the above **RandomForest** model and print them (3 marks)

In [189]:
# Top 3 features for RandomForest
# Enter your code here

important_features = rf_model.feature_importances_
features_name = data.iloc[:,:9].columns
imp_feature_frame = pd.DataFrame({'features': features_name, 'importance': important_features}).sort_values(by = "importance", ascending =False).iloc[0:3,:]
print(imp_feature_frame)
print(list(imp_feature_frame.features))


                   features  importance
2  Uniformity of Cell Shape    0.345950
5           Bland Chromatin    0.306016
1  Uniformity of Cell Size     0.178360
['Uniformity of Cell Shape', 'Bland Chromatin', 'Uniformity of Cell Size ']


### 5. Boosting: (7 marks)
#### a) Build an AdaBoost model with training data and print the accuracy (4 marks)
---

Hyperparameters:

Base estimator = DecisionTreeClassifier, max_depth = 4

n_estimators = 200

random_state = 42

learning_rate = 0.05


---









In [190]:
# AdaBoost Classification
# Enter your code here

base_dt = DecisionTreeClassifier(max_depth =4)
ada_boost_model = AdaBoostClassifier(base_estimator= base_dt, n_estimators=200, random_state=42, learning_rate=.05)
ada_boost_model.fit(X_train, y_train)
ada_predict = ada_boost_model.predict(X_test)
print("AdaBoost Classification's Accuracy is:", accuracy_score(y_test, ada_predict))


AdaBoost Classification's Accuracy is: 0.9523809523809523


#### b) Calculate the top 3 important features for the above **AdaBoost** model and print them (3 marks)

In [191]:
# Top 3 features for AdaBoost
# Enter your code here

important_features2 = ada_boost_model.feature_importances_
features_name = data.iloc[:,:9].columns
imp_feature_frame2 = pd.DataFrame({'features': features_name, 'importance': important_features2}).sort_values(by = "importance", ascending =False).iloc[0:3,:]
print(imp_feature_frame2)
print(list(imp_feature_frame.features))

                   features  importance
6               Bare Nuclei    0.353421
2  Uniformity of Cell Shape    0.139991
1  Uniformity of Cell Size     0.137421
['Uniformity of Cell Shape', 'Bland Chromatin', 'Uniformity of Cell Size ']


### 6. Voting : Using a voting classifier, build an ensemble of RandomForestClassifier, DecisionTreeClassifier, Support Vector Machine and Logistic Regression. (7 marks)


---


Use max_depth = 4, n_estimators = 200, voting = soft

In [192]:
# Voting Ensemble for Classification
# Enter your code here
dt = DecisionTreeClassifier(max_depth = 4)
rf = RandomForestClassifier( n_estimators = 200)
svm = SVC(probability=True)
lr= LogisticRegression()
classifiers = [("RandomForestClassifier", rf), ("DecisionTreeClassifier", dt), ("Support Vector Machine",svm),("Logistic Regression", lr) ]
vc = VotingClassifier(estimators=classifiers, voting = "soft")
vc.fit(X_train, y_train)
vc_pred = vc.predict(X_test)
vc_Model_accuracy = accuracy_score(y_test,vc_pred)
print("Voting Classifer Accuracy is:", vc_Model_accuracy)

Voting Classifer Accuracy is: 0.9619047619047619


### 7. Mention the best model among the above 4 models and its accuracy (1 mark)

In [196]:
# Write your answer here
dict_accuracy = {"Model":["Voting Classifier","AdaBoost Classifier","Random Forest Classifier","Bagging Classifier"],
                 "Accuracy":[vc_Model_accuracy,accuracy_score(y_test, ada_predict),rf_Model_accuracy,Bagging_Model_accuracy]}

df_acc = pd.DataFrame(dict_accuracy).sort_values(by="Accuracy",ascending= False)

print("The best model among the above 4 models is Voting Classifer model and its accuracy score is:", vc_Model_accuracy)
df_acc

The best model among the above 4 models is Voting Classifer model and its accuracy score is: 0.9619047619047619


,Model,Accuracy
0,Voting Classifier,0.961905
1,AdaBoost Classifier,0.952381
2,Random Forest Classifier,0.952381
3,Bagging Classifier,0.947619
